In [2]:
import argparse
import cv2
import numpy as np
import SimpleITK as sitk
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models.vgg import vgg19
import numpy as np
import matplotlib.pyplot as plt
from keras import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.datasets import mnist
from keras.models import load_model
import os
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import load_img, img_to_array
from skimage.transform import resize
import pywt
import pywt.data
import pydicom
from PIL import Image
import nrrd
from lungmask import LMInferer

### 0. Helper Functions

In [3]:
class VGG19(torch.nn.Module):
    def __init__(self, device='cpu'):
        super(VGG19, self).__init__()
        features = list(vgg19(pretrained=True).features)
        if device == "cuda":
            self.features = nn.ModuleList(features).cuda().eval()
        else:
            self.features = nn.ModuleList(features).eval()

    def forward(self, x):
        feature_maps = []
        for idx, layer in enumerate(self.features):
            x = layer(x)
            if idx == 3:
                feature_maps.append(x)
        return feature_maps

In [7]:
class Fusion:
    def __init__(self, input):
        """
        Class Fusion constructor

        Instance Variables:
            self.images: input images
            self.model: CNN model, default=vgg19
            self.device: either 'cuda' or 'cpu'
        """
        self.input_images = input
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = VGG19(self.device)

    def fuse(self):
        """
        A top level method which fuse self.images
        """
        # Convert all images to YCbCr format
        self.normalized_images = [-1 for img in self.input_images]
        self.YCbCr_images = [-1 for img in self.input_images]
        for idx, img in enumerate(self.input_images):
            if not self._is_gray(img):
                self.YCbCr_images[idx] = self._RGB_to_YCbCr(img)
                self.normalized_images[idx] = self.YCbCr_images[idx][:, :, 0]
            else:
                self.normalized_images[idx] = img / 255.
        # Transfer all images to PyTorch tensors
        self._tranfer_to_tensor()
        # Perform fuse strategy
        fused_img = self._fuse()[:, :, 0]
        # Reconstruct fused image given rgb input images
        for idx, img in enumerate(self.input_images):
            if not self._is_gray(img):
                self.YCbCr_images[idx][:, :, 0] = fused_img
                fused_img = self._YCbCr_to_RGB(self.YCbCr_images[idx])
                fused_img = np.clip(fused_img, 0, 1)

        return (fused_img * 255).astype(np.uint8)
        # return fused_img

    def _fuse(self):
        """
        Perform fusion algorithm
        """
        with torch.no_grad():

            imgs_sum_maps = [-1 for tensor_img in self.images_to_tensors]
            for idx, tensor_img in enumerate(self.images_to_tensors):
                imgs_sum_maps[idx] = []
                feature_maps = self.model(tensor_img)
                for feature_map in feature_maps:
                    sum_map = torch.sum(feature_map, dim=1, keepdim=True)
                    imgs_sum_maps[idx].append(sum_map)

            max_fusion = None
            for sum_maps in zip(*imgs_sum_maps):
                features = torch.cat(sum_maps, dim=1)
                weights = self._softmax(F.interpolate(features,
                                        size=self.images_to_tensors[0].shape[2:]))
                weights = F.interpolate(weights,
                                        size=self.images_to_tensors[0].shape[2:])
                current_fusion = torch.zeros(self.images_to_tensors[0].shape)
                for idx, tensor_img in enumerate(self.images_to_tensors):
                    current_fusion += tensor_img * weights[:,idx]
                if max_fusion is None:
                    max_fusion = current_fusion
                else:
                    max_fusion = torch.max(max_fusion, current_fusion)

            output = np.squeeze(max_fusion.cpu().numpy())
            if output.ndim == 3:
                output = np.transpose(output, (1, 2, 0))
            return output
        
        
    def _RGB_to_YCbCr(self, img_RGB):
            """
            A private method which converts an RGB image to YCrCb format
            """
            img_RGB = img_RGB.astype(np.float32) / 255.
            return cv2.cvtColor(img_RGB, cv2.COLOR_RGB2YCrCb)

    def _YCbCr_to_RGB(self, img_YCbCr):
            """
            A private method which converts a YCrCb image to RGB format
            """
            img_YCbCr = img_YCbCr.astype(np.float32)
            return cv2.cvtColor(img_YCbCr, cv2.COLOR_YCrCb2RGB)

    def _is_gray(self, img):
            """
            A private method which returns True if image is gray, otherwise False
            """
            if len(img.shape) < 3:
                return True
            if img.shape[2] == 1:
                return True
            b, g, r = img[:,:,0], img[:,:,1], img[:,:,2]
            if (b == g).all() and (b == r).all():
                return True
            return False

    def _softmax(self, tensor):
            """
            A private method which compute softmax ouput of a given tensor
            """
            tensor = torch.exp(tensor)
            tensor = tensor / tensor.sum(dim=1, keepdim=True)
            return tensor

    def _tranfer_to_tensor(self):
            """
            A private method to transfer all input images to PyTorch tensors
            """
            self.images_to_tensors = []
            for image in self.normalized_images:
                np_input = image.astype(np.float32)
                if np_input.ndim == 2:
                    np_input = np.repeat(np_input[None, None], 3, axis=1)
                else:
                    np_input = np.transpose(np_input, (2, 0, 1))[None]
                if self.device == "cuda":
                    self.images_to_tensors.append(torch.from_numpy(np_input).cuda())
                else:
                    self.images_to_tensors.append(torch.from_numpy(np_input)) 

In [4]:
def manipulating_nrrd_contrast(img, level):
    img_c = img.astype(int).copy()
    factor = (8 * (level+255)) / (255 * (259-level)) 
    img_c = factor * (img_c - 128) + 128
    img_c = np.clip(img_c, 0, 255)
    return img_c.astype(np.uint8)

### 1. Convert NRRD files into image slices

In [5]:
PID = { 'R01-005','R01-012','R01-013','R01-014','R01-017','R01-021','R01-026','R01-027','R01-028','R01-029',
        'R01-038','R01-043','R01-046','R01-048','R01-049','R01-051','R01-052','R01-054','R01-055','R01-056',
        'R01-057','R01-059','R01-060','R01-061','R01-062','R01-063','R01-064','R01-065','R01-066','R01-067',
        'R01-068','R01-069','R01-071','R01-072','R01-073','R01-076','R01-078','R01-080','R01-081','R01-083',
        'R01-084','R01-089','R01-091','R01-093','R01-094','R01-096','R01-097','R01-098','R01-100','R01-101',
        'R01-102','R01-103','R01-104','R01-105','R01-106','R01-107','R01-108','R01-109','R01-110','R01-111',
        'R01-112','R01-113','R01-114','R01-115','R01-116','R01-117','R01-118','R01-119','R01-120','R01-121',
        'R01-122','R01-123','R01-124','R01-125','R01-126','R01-127','R01-128','R01-129','R01-130','R01-131',
        'R01-132','R01-133','R01-134','R01-135','R01-136','R01-138','R01-139','R01-140','R01-141','R01-142',
        'R01-144','R01-145','R01-146','R01-147','R01-148','R01-149','R01-151','R01-152','R01-154','R01-156',
        'R01-157','R01-158','R01-159','R01-160'}

In [30]:
print(final)

<PIL.Image.Image image mode=L size=262144x1 at 0x2B428FA30>


In [25]:
# Specify the path where you want to create the folders
path = "./Dataset Lung img"
# Make sure the path exists, if not, create it
os.makedirs(path, exist_ok=True)

# Loop through the PID list and create a folder for each name
for folder_name in PID:
    os.makedirs(os.path.join(path, folder_name), exist_ok=True)

In [26]:
#load de-noising model
model = load_model('autoencoder_denoise_model.h5')

In [35]:
for p in PID:
    print("--------------------", p, "--------------------")
    #ensure reference is the same in CT and PET scans
    ct_image = sitk.ReadImage(f'/Users/salma/Downloads/NRRD Dataset/{p}_CT.nrrd')
    pt_image = sitk.ReadImage(f'/Users/salma/Downloads/NRRD Dataset/{p}_PT.nrrd')

    slices = []
    #ensure proper image registration
    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(ct_image)  # Use CT image as the reference
    resampler.SetInterpolator(sitk.sitkLinear)  # Choose an interpolation method
    resized_image = resampler.Execute(pt_image)
    sitk.WriteImage(resized_image, f"/Users/salma/Downloads/NRRD Dataset/{p}_PT_resized.nrrd")

    #convert CT nrrd to images
    filename = f'/Users/salma/Downloads/NRRD Dataset/{p}_CT.nrrd'
    readdata, header = nrrd.read(filename)

    volume_path = f'/Users/salma/Downloads/NRRD Dataset/{p}_CT.nrrd'
    image = sitk.ReadImage(volume_path)
    original_spacing = image.GetSpacing()
    original_size = image.GetSize()
    new_spacing = (original_spacing[0], original_spacing[1], (original_spacing[2]*original_size[2])/128.0)
    new_size = (original_size[0], original_size[1], 128)
    resampled_image = sitk.Resample(image, new_size, sitk.Transform(), sitk.sitkLinear, image.GetOrigin(), new_spacing, image.GetDirection())
    image_array = sitk.GetArrayFromImage(resampled_image)

    inferer = LMInferer()
    segmentation = inferer.apply(image_array)  # default model is U-net(R231)

    for frame in range(min(segmentation.shape[0], image.GetSize()[2])):
        pixel_count = segmentation[frame].sum()
        if pixel_count > 0:
            slices.append(frame)
            # Extract a 2D slice from the SimpleITK image
            slice_sitk = image[:,:,frame]
            # Convert the SimpleITK image slice to a numpy array and ensure it's of integer type
            img_arr = sitk.GetArrayFromImage(slice_sitk).astype(int)
            b = np.asarray(img_arr).astype(int)
            final = Image.fromarray(manipulating_nrrd_contrast(b,128))
            #final = final.rotate(270, expand=True)
            final.save(f'./Dataset Lung img/{p}/{p}_{frame}_CT.jpg')
    
    # Define a threshold value for binarization
    #threshold_value = 150

    #convert PT nrrd to images
    filename = f'/Users/salma/Downloads/NRRD Dataset/{p}_PT_resized.nrrd'
    readdata, header = nrrd.read(filename)
    for i in range(readdata.shape[2]):
        if i in slices:
            b = np.asarray(readdata[:,:,i]).astype(np.uint8)
            # Use Gaussian blur to smooth out the image and reduce noise
            #blurred = cv2.GaussianBlur(b, (5, 5), 0)
            # Binarize the image using a threshold
            #_, binary = cv2.threshold(blurred, threshold_value, 255, cv2.THRESH_BINARY)
            final = Image.fromarray(b)
            final = final.rotate(270, expand=True)

            result = cv2.normalize(np.array(final), dst=None, alpha=0, beta=255,norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
            cv2.imwrite(f'./Dataset Lung img/{p}/{p}_{i}_PT.jpg', result)
            
    

            #apply de-noising
            testing_image = f'./Dataset Lung img/{p}/{p}_{i}_PT.jpg'
            image_size = (128, 128)

            img = load_img(testing_image, target_size=image_size)
            img_array = img_to_array(img)
            img_array_test = img_array.astype('float32') / 255.0
            grayscale_images = np.mean(img_array_test, axis=-1, keepdims=True)
            img_array_test = grayscale_images.reshape(1, 128, 128, 1)

            pred = model.predict(img_array_test)
            # Assuming pred is already defined and is a 2D array with shape (128, 128)
            image_data = pred.reshape(128, 128)

            # Convert the data to an 8-bit format (0-255)
            image_data = (image_data * 255).astype(np.uint8)

            # Create an image from the array data
            img = Image.fromarray(image_data, 'L')  # 'L' indicates grayscale mode

            

            # Save the image to a file
            img.save(f'./Dataset Lung img/{p}/{p}_{i}_PT_denoised.jpg')

-------------------- R01-103 --------------------
lungmask 2023-11-01 16:38:08 No GPU found, using CPU instead


7it [01:37, 13.86s/it]                         

lungmask 2023-11-01 16:39:47 Postprocessing



100%|██████████| 74/74 [00:01<00:00, 71.02it/s]


1/1 [==============================] - 0s 20ms/step
-------------------- R01-062 --------------------
lungmask 2023-11-01 16:39:58 No GPU found, using CPU instead


...

1/1 [==============================] - 0s 22ms/step
-------------------- R01-059 --------------------
lungmask 2023-11-01 20:09:04 No GPU found, using CPU instead


7it [01:32, 13.27s/it]                         

lungmask 2023-11-01 20:10:38 Postprocessing



100%|██████████| 18/18 [00:00<00:00, 98.41it/s] 


1/1 [==============================] - 0s 19ms/step


### 2. Fuse Images Together

In [8]:
import re
count = 0
PID = {'R01-043'}
for p in PID:
    count += 1
    print(f'Processing {count} out of {len(PID)}')
    # Define the directory path where the images are stored
    directory_path = f'./Lung Img/{p}/'
    #filename = f'./NRRD Dataset/{p}_CT.nrrd'
    #readdata, header = nrrd.read(filename)

    slices_num = []

    # Get list of all files in the directory
    all_files = os.listdir(directory_path)

    for filename in all_files:
        # Check if filename starts with "p_" and ends with "_CT.jpg"
        if filename.startswith(f'{p}_') and filename.endswith("_CT.jpg"):
            # Extract frame number using regular expression
            match = re.search(rf"{p}_(\d+)_CT\.jpg", filename)
            if match:
                slices_num.append(int(match.group(1)))

    for i in slices_num:
        input_images = []
        
        ct = cv2.imread(f'./Lung Img/{p}/{p}_{i}_CT.jpg')
        ct = cv2.normalize(ct, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)

        pt = cv2.imread(f'./Lung Img/{p}/{p}_{i}_PT_denoised.jpg')
        # Convert the numpy array to a PIL Image
        image = Image.fromarray(pt)
        # Upscale using bilinear interpolation
        upscaled_image = cv2.resize(pt, (512,512))
        # Convert the PIL Image back to a numpy array
        pt = np.array(upscaled_image)
        #pt = cv2.normalize(pt, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)

        input_images.append(ct)
        input_images.append(pt)
        
        # Compute fusion image
        FU = Fusion(input_images)
        fusion_img = FU.fuse()

        fu_image = Image.fromarray(fusion_img)
        downscaled_image = fu_image.resize((128, 128), Image.BILINEAR)
        fused_img = np.array(downscaled_image)

        # Write fusion image
        cv2.imwrite(f'./Fused Lung 2/{p}_{i}_Fused_2.jpg', fused_img)

Processing 1 out of 104
Processing 2 out of 104
Processing 3 out of 104
Processing 4 out of 104
Processing 5 out of 104
Processing 6 out of 104
Processing 7 out of 104
Processing 8 out of 104
Processing 9 out of 104
Processing 10 out of 104
Processing 11 out of 104
Processing 12 out of 104
Processing 13 out of 104
Processing 14 out of 104
Processing 15 out of 104
Processing 16 out of 104
Processing 17 out of 104
Processing 18 out of 104
Processing 19 out of 104
Processing 20 out of 104
Processing 21 out of 104
Processing 22 out of 104
Processing 23 out of 104
Processing 24 out of 104
Processing 25 out of 104
Processing 26 out of 104
Processing 27 out of 104
Processing 28 out of 104
Processing 29 out of 104
Processing 30 out of 104
Processing 31 out of 104
Processing 32 out of 104
Processing 33 out of 104
Processing 34 out of 104
Processing 35 out of 104
Processing 36 out of 104
Processing 37 out of 104
Processing 38 out of 104
Processing 39 out of 104
Processing 40 out of 104
Processin